<a href="https://colab.research.google.com/github/jong104b-kr/AIFFEL_quest_cr/blob/master/dlthonm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir ~/aiffel/jellyfish

In [3]:
!pip install kaggle

In [4]:
mkdir ~/.kaggle

In [5]:
mv kaggle.json ~/.kaggle/kaggle.json

In [7]:
cd ~/aiffel/jellyfish

'/root/aiffel/jellyfish'

In [8]:
!kaggle datasets download -d anshtanwar/jellyfish-types

Dataset URL: https://www.kaggle.com/datasets/anshtanwar/jellyfish-types
License(s): Attribution 4.0 International (CC BY 4.0)


In [9]:
!unzip jellyfish-types.zip

Archive:  jellyfish-types.zip
  inflating: Moon_jellyfish/01.jpg   
  inflating: Moon_jellyfish/02.jpg   
  inflating: Moon_jellyfish/03.jpg   
  inflating: Moon_jellyfish/04.jpg   
  inflating: Moon_jellyfish/05.jpg   
  inflating: Moon_jellyfish/06.jpg   
  inflating: Moon_jellyfish/08.jpg   
  inflating: Moon_jellyfish/10.jpg   
  inflating: Moon_jellyfish/12.jpg   
  inflating: Moon_jellyfish/13.jpg   
  inflating: Moon_jellyfish/14.jpg   
  inflating: Moon_jellyfish/16.jpg   
  inflating: Moon_jellyfish/17.jpg   
  inflating: Moon_jellyfish/18.jpg   
  inflating: Moon_jellyfish/21.jpg   
  inflating: Moon_jellyfish/22.jpg   
  inflating: Moon_jellyfish/24.jpg   
  inflating: Moon_jellyfish/25.jpg   
  inflating: Moon_jellyfish/26.jpg   
  inflating: Moon_jellyfish/28.jpg   
  inflating: Moon_jellyfish/29.JPG   
  inflating: Moon_jellyfish/30.jpg   
  inflating: Moon_jellyfish/31.jpg   
  inflating: Moon_jellyfish/32.jpg   
  inflating: Moon_jellyfish/34.jpg   
  inflating: Moon_je

In [ ]:
!pip install wandb

In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.integration.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import glob
import os

In [12]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape( -1, 28, 28, 1)
X_test = X_test.reshape( -1, 28, 28, 1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

# 시각화를 위해 sample 데이터를 준비합니다.
X_sample = X_test[:50]
y_sample = y_test[:50]

CLASS_NAMES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
wandb.login(key = "67a92f37cee1164f4de54893974e616314465bf4")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jong104b (jong104b-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [19]:
sweep_config = {
    "name": "sweep_test_core",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }

        }
    }

In [20]:
def train():
    default_config = {
        "input" : (28,28,1),
        "filter" : 16,
        "kernel" : (3,3),
        "activation" : "relu",
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(config.filter, config.kernel, activation=config.activation, input_shape=config.input))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)

    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.

    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback(validation_data = (X_sample, y_sample),
                                        lables = CLASS_NAMES,
                                        predictions = 10,
                                        input_type = "images")])

    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.

    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

In [21]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'jong104b-home',
                       project = 'dlthonm3')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: kpm83etp
Sweep URL: https://wandb.ai/jong104b-home/dlthonm3/sweeps/kpm83etp


wandb: Agent Starting Run: 9ta1uy1k with config:
wandb: 	epoch: 9
wandb: 	learning_rate: 0.05588051009021565


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/9


Run 9ta1uy1k errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-20-3858a0483ca4>", line 40, in train
    model.fit(X_train, y_train,
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.11/dist-packages/wandb/integration/keras/keras.py", line 668, in on_train_batch_end
    wandb.run.summary["graph"] = wandb.Graph.from_keras(self.model)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/data_types/graph.py", line 357, in from_keras
    for in_layer in _nest(in_node.inbound_layers):
                          ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Node' object has no attribute 'inbound_layers'

wandb: ERROR Run 9ta1uy1k errored:
wandb: ERROR Trac

Epoch 1/5


Run usyo4a6m errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-20-3858a0483ca4>", line 40, in train
    model.fit(X_train, y_train,
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.11/dist-packages/wandb/integration/keras/keras.py", line 668, in on_train_batch_end
    wandb.run.summary["graph"] = wandb.Graph.from_keras(self.model)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/data_types/graph.py", line 357, in from_keras
    for in_layer in _nest(in_node.inbound_layers):
                          ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Node' object has no attribute 'inbound_layers'

wandb: ERROR Run usyo4a6m errored:
wandb: ERROR Trac

Epoch 1/5


Run 2fa3lbt0 errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-20-3858a0483ca4>", line 40, in train
    model.fit(X_train, y_train,
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.11/dist-packages/wandb/integration/keras/keras.py", line 668, in on_train_batch_end
    wandb.run.summary["graph"] = wandb.Graph.from_keras(self.model)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/data_types/graph.py", line 357, in from_keras
    for in_layer in _nest(in_node.inbound_layers):
                          ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Node' object has no attribute 'inbound_layers'

wandb: ERROR Run 2fa3lbt0 errored:
wandb: ERROR Trac

In [22]:
pwd

'/root/aiffel/jellyfish'

In [ ]:
# 케창딥 p.298 코드 8-6 응용, 이미지를 훈련,검증,테스트 디렉터리로 복사
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("test")

def make_subset(subset_name, start_index, end_index):
    for category in ("Moon_jellyfish", "barrel_jellyfish", "blue_jellyfish", "compass_jellyfish", "lions_mane_jellyfish", "mauve_stinger_jellyfish"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=10)
make_subset("validation", start_index=10, end_index=15)
make_subset("test", start_index=15, end_index=25)

In [25]:
# 케창딥 p.299 강아지vs고양이 분류를 위한 소규모 컨브넷 만들기
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180,180,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32,kernel_size=3,activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64,kernel_size=3,activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128,kernel_size=3,activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256,kernel_size=3,activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256,kernel_size=3,activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model = keras.Model(inputs=inputs,outputs=outputs)

In [26]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 178, 178, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 89, 89, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 87, 87, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 43, 43, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 41, 41, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 18, 18, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 9, 9, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 7, 7, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │          12,545 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 991,041 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# 케창딥 p.301 코드 8-8 모델 훈련 설정하기
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [28]:
# 케창딥 p.302 코드 8-9 image_dataset_from_directory를 사용하여 이미지 읽기
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train", # 경로 수정하기
    image_size = (180,180),
    batch_size = 32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation", # 경로 수정하기
    image_size = (180,180),
)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test", # 경로 수정하기
    image_size = (180,180),
    batch_size = 32
)

Found 0 files belonging to 1 classes.


ValueError: No images found in directory test/train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [29]:
# p.304 코드 8-10 Dataset이 반환하는 데이터와 레이블 크기 확인하기
for data_batch, labels_batch in train_dataset:
  print("데이터 배치 크기:", data_batch_shape)
  print("레이블 배치 크기:", labels_batch.shape)
  break

NameError: name 'train_dataset' is not defined

In [30]:
# p.304 코드 8-11 Dataset을 사용하여 모델 훈련하기
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = "convert_from_scrach.h5",
        save_best_only = True,
        monitor = "val_loss")
]

history = model.fit(
    train_dataset,
    epochs = 30,
    validation_data = validation_dataset,
    callbacks = callbacks
)

NameError: name 'train_dataset' is not defined

In [31]:
# p.305 코드 8-12 훈련 과정의 정확도와 손실 그래프 그리기
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [32]:
# p.306 코드 8-13 테스트 세트에서 모델 평가하기
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

ValueError: File not found: filepath=convnet_from_scratch.keras. Please ensure the file is an accessible `.keras` zip file.

In [33]:
# p.307 코드 8-14 컨브넷에 추가할 데이터 증식 단계 정의하기
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [34]:
# p.307 코드 8-15 랜덤하게 증식된 훈련 이미지 출력하기
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

NameError: name 'train_dataset' is not defined

<Figure size 1000x1000 with 0 Axes>

In [35]:
# p.309 이미지 증식과 드롭아웃을 포함한 컨브넷 만들기
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [36]:
# p.309 코드 8-17 규제를 추가한 컨브넷 훈련하기
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

NameError: name 'train_dataset' is not defined

In [37]:
# p.310 코드 8-18 테스트 세트에서 모델 훈련하기
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

ValueError: File not found: filepath=convnet_from_scratch_with_augmentation.keras. Please ensure the file is an accessible `.keras` zip file.

In [38]:
# p.313 코드 8-19 VGG16 합성곱 기반 층 만들기
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [39]:
# 데이터 증식을 사용하지 않는 빠른 특성 추출
# p.316 코드 8-20 VGG16 특성과 해당 레이블 추출하기
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

NameError: name 'train_dataset' is not defined

In [40]:
# p.317 코드 8-21 밀집 연결 분류기 정의하고 훈련하기
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

NameError: name 'train_features' is not defined

In [41]:
# p.318 코드 8-22 결과를 그래프로 나타내기
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [42]:
# p.319 코드 8-23 VGG16 합성곱 기반 층을 만들고 동결하기
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

In [43]:
# p.319 코드 8-24 동결하기 전과 후에 훈련 가능한 가중치 리스트 출력하기
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights before freezing the conv base: 26
This is the number of trainable weights after freezing the conv base: 0


In [44]:
# p.319 코드 8-25 데이터 증식 단계와 밀집 분류기를 합성곱 기반 층에 추가하기
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [46]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

NameError: name 'train_dataset' is not defined

In [47]:
# p.321 코드 8-26 테스트 세트에서 모델 평가하기
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

ValueError: File not found: filepath=feature_extraction_with_data_augmentation.keras. Please ensure the file is an accessible `.keras` zip file.

In [48]:
# p.325 코드 8-27 마지막에서 네 번째 층까지 모든 층 동결하기
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [49]:
# p.326 코드 8-28 모델 미세 조정하기
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

NameError: name 'train_dataset' is not defined